### AML Alert Prioritization and Risk Scoring System

##### Objective

This project implements a two stage, risk based Anti Money Laundering monitoring framework aligned with common banking practices and CAMS typologies. The system mirrors how financial institutions generate scenario based alerts at the transaction level and apply model driven risk scoring to prioritize alerts for investigation.

##### Intended Use

The model is designed to support alert triage and investigator prioritization. It does not automate SAR decisions and is intended to operate with human oversight in accordance with regulatory expectations.

##### Dataset

Synthetic AML transaction data with embedded laundering typologies. The dataset is used strictly for research and demonstration purposes.

In [5]:
import pandas as pd
import numpy as np

df = pd.read_csv("data/SAML-D.csv")

df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_")
)

df.head()


,time,date,sender_account,receiver_account,amount,payment_currency,received_currency,sender_bank_location,receiver_bank_location,payment_type,is_laundering,laundering_type
0,10:35:19,2022-10-07,8724731955,2769355426,1459.15,UK pounds,UK pounds,UK,UK,Cash Deposit,0,Normal_Cash_Deposits
1,10:35:20,2022-10-07,1491989064,8401255335,6019.64,UK pounds,Dirham,UK,UAE,Cross-border,0,Normal_Fan_Out
2,10:35:20,2022-10-07,287305149,4404767002,14328.44,UK pounds,UK pounds,UK,UK,Cheque,0,Normal_Small_Fan_Out
3,10:35:21,2022-10-07,5376652437,9600420220,11895.00,UK pounds,UK pounds,UK,UK,ACH,0,Normal_Fan_In
4,10:35:21,2022-10-07,9614186178,3803336972,115.25,UK pounds,UK pounds,UK,UK,Cash Deposit,0,Normal_Cash_Deposits


### Timestamp Normalization

In [6]:
df["transaction_datetime"] = pd.to_datetime(
    df["date"].astype(str) + " " + df["time"].astype(str),
    errors="coerce"
)

df = df.sort_values("transaction_datetime").reset_index(drop=True)

df[["transaction_datetime", "sender_account", "receiver_account", "amount"]].head()


,transaction_datetime,sender_account,receiver_account,amount
0,2022-10-07 10:35:19,8724731955,2769355426,1459.15
1,2022-10-07 10:35:20,1491989064,8401255335,6019.64
2,2022-10-07 10:35:20,287305149,4404767002,14328.44
3,2022-10-07 10:35:21,5376652437,9600420220,11895.00
4,2022-10-07 10:35:21,9614186178,3803336972,115.25


### Validation Checks

In [7]:
df.info()
df.isna().sum().sort_values(ascending=False).head(10)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9504852 entries, 0 to 9504851
Data columns (total 13 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   time                    object        
 1   date                    object        
 2   sender_account          int64         
 3   receiver_account        int64         
 4   amount                  float64       
 5   payment_currency        object        
 6   received_currency       object        
 7   sender_bank_location    object        
 8   receiver_bank_location  object        
 9   payment_type            object        
 10  is_laundering           int64         
 11  laundering_type         object        
 12  transaction_datetime    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(3), object(8)
memory usage: 942.7+ MB


time                      0
date                      0
sender_account            0
receiver_account          0
amount                    0
payment_currency          0
received_currency         0
sender_bank_location      0
receiver_bank_location    0
payment_type              0
dtype: int64

In [10]:
df["payment_type"].value_counts().head(10)

Credit card        2012909
Debit card         2012103
Cheque             2011419
ACH                2008807
Cross-border        933931
Cash Withdrawal     300477
Cash Deposit        225206
Name: payment_type, dtype: int64

In [9]:
df["sender_bank_location"].value_counts().head(10)

UK             9183088
Turkey           20902
Switzerland      20503
Pakistan         20346
UAE              20081
Nigeria          20027
Spain            19391
Germany          19259
USA              19027
Italy            18895
Name: sender_bank_location, dtype: int64

### Create New Features

In [11]:
import pandas as pd
import numpy as np

df = df.sort_values(["sender_account", "transaction_datetime"]).copy()

df["is_cross_border"] = (df["sender_bank_location"] != df["receiver_bank_location"]).astype("int8")
df["is_cash_like"] = df["payment_type"].astype(str).str.contains("Cash", case=False, na=False).astype("int8")

df["txn_count"] = 1
df[["sender_account","receiver_account","amount","is_cross_border","is_cash_like","transaction_datetime"]].head()


,sender_account,receiver_account,amount,is_cross_border,is_cash_like,transaction_datetime
8572082,9018,2388293593,3319.06,1,0,2023-07-22 09:51:28
3210514,28511,3072405466,6371.25,0,0,2023-01-24 23:28:15
4191567,28511,3072405466,3878.00,0,0,2023-02-24 23:31:38
5018226,28511,3072405466,4109.92,0,0,2023-03-24 20:51:59
5938108,28511,3072405466,7147.58,0,0,2023-04-24 19:38:10


Stage 1: Scenario Based Transaction Monitoring

Financial institutions typically begin AML monitoring with scenario based rules that identify behaviors associated with known money laundering typologies. These scenarios prioritize coverage and interpretability over precision.

In this stage, transaction level metrics are computed using rolling time windows to detect:

Transaction velocity

Cross border activity bursts

Cash intensive behavior

Rapid transaction patterns

An alert is generated when one or more scenarios are triggered for an account on a given day. Alerts are grouped at the sender account and day level to reflect how cases are typically created for investigation.

## Stage 1: Scenario based monitoring to generate alerts
1) Build rolling window features per sender
* We’ll compute 1 day and 7 day windows
* note: In real world we would add a 30 day window on top

In [13]:
df = df.sort_values(["sender_account", "transaction_datetime"]).copy()

df["txn_count"] = 1

def add_fast_rolling(g):
    g = g.set_index("transaction_datetime")

    # 1d and 7d windows
    g["cnt_1d"] = g["txn_count"].rolling("1D").sum()
    g["cnt_7d"] = g["txn_count"].rolling("7D").sum()

    g["amt_1d"] = g["amount"].rolling("1D").sum()
    g["amt_7d"] = g["amount"].rolling("7D").sum()

    g["xborder_cnt_7d"] = g["is_cross_border"].rolling("7D").sum()
    g["cash_cnt_7d"] = g["is_cash_like"].rolling("7D").sum()

    g["mins_since_prev"] = g.index.to_series().diff().dt.total_seconds().div(60)

    return g.reset_index()

df_feat = df.groupby("sender_account", group_keys=False).apply(add_fast_rolling)

df_feat.head()


,transaction_datetime,time,date,sender_account,receiver_account,amount,payment_currency,received_currency,sender_bank_location,receiver_bank_location,...,is_cross_border,is_cash_like,txn_count,cnt_1d,cnt_7d,amt_1d,amt_7d,xborder_cnt_7d,cash_cnt_7d,mins_since_prev
0,2023-07-22 09:51:28,09:51:28,2023-07-22,9018,2388293593,3319.06,UK pounds,Euro,UK,Germany,...,1,0,1,1.0,1.0,3319.06,3319.06,1.0,0.0,NaN
0,2023-01-24 23:28:15,23:28:15,2023-01-24,28511,3072405466,6371.25,UK pounds,UK pounds,UK,UK,...,0,0,1,1.0,1.0,6371.25,6371.25,0.0,0.0,NaN
1,2023-02-24 23:31:38,23:31:38,2023-02-24,28511,3072405466,3878.00,UK pounds,UK pounds,UK,UK,...,0,0,1,1.0,1.0,3878.00,3878.00,0.0,0.0,44643.383333
2,2023-03-24 20:51:59,20:51:59,2023-03-24,28511,3072405466,4109.92,UK pounds,UK pounds,UK,UK,...,0,0,1,1.0,1.0,4109.92,4109.92,0.0,0.0,40160.350000
3,2023-04-24 19:38:10,19:38:10,2023-04-24,28511,3072405466,7147.58,UK pounds,UK pounds,UK,UK,...,0,0,1,1.0,1.0,7147.58,7147.58,0.0,0.0,44566.183333


### The Scenario Rules

In [17]:
# Scenario rules

df_feat["scn_high_velocity"] = (df_feat["cnt_1d"] >= 15).astype("int8")

df_feat["scn_xborder_burst"] = (df_feat["xborder_cnt_7d"] >= 5).astype("int8")

df_feat["scn_cash_intensive"] = (df_feat["cash_cnt_7d"] >= 5).astype("int8")

df_feat["scn_rapid_activity"] = (
    df_feat["mins_since_prev"].fillna(9999) <= 2
).astype("int8")

scenario_cols = [
    "scn_high_velocity",
    "scn_xborder_burst",
    "scn_cash_intensive",
    "scn_rapid_activity",
]

df_feat["scenario_hits"] = df_feat[scenario_cols].sum(axis=1).astype("int8")

df_feat["any_alert"] = (df_feat["scenario_hits"] > 0).astype("int8")

df_feat[scenario_cols + ["scenario_hits", "any_alert"]].mean()


scn_high_velocity     0.225845
scn_xborder_burst     0.180662
scn_cash_intensive    0.003053
scn_rapid_activity    0.072460
scenario_hits         0.482020
any_alert             0.268343
dtype: float64

Stage 2: Alert Risk Scoring and Prioritization

Scenario based monitoring typically produces more alerts than investigators can reasonably review. To address this, a second stage model is applied to prioritize alerts by risk.

In this stage:

Alerts generated in Stage 1 are enriched with aggregated behavioral features

A supervised model predicts the likelihood that an alert corresponds to laundering activity

Alerts are ranked by risk score to optimize investigator focus

Both a logistic regression baseline and a gradient boosted model are evaluated to balance interpretability and performance.

#### Stage 2A: Alert Aggregation and Labeling
* Group transactions into alerts
* Assign alert outcomes using is_laudering
* This is where your "alert label is 1 if any transaction..." logic lives

#### Create alert_date before alert aggregation

In [18]:
df_feat["alert_date"] = df_feat["transaction_datetime"].dt.date

### Fan-out during alert aggregation

Instead of rolling unique receivers per transaction, do it per alert day, which is exactly how banks analyze it.

Later, in alert aggregation:

In [21]:
df_feat["alert_date"] = df_feat["transaction_datetime"].dt.date

alerts = (
    df_feat[df_feat["any_alert"] == 1]
    .groupby(["sender_account", "alert_date"], as_index=False)
    .agg(
        alert_txn_count=("txn_count", "sum"),
        alert_amt_sum=("amount", "sum"),
        alert_unique_receivers=("receiver_account", "nunique"),
        alert_xborder_cnt=("is_cross_border", "sum"),
        alert_cash_cnt=("is_cash_like", "sum"),
        max_cnt_1d=("cnt_1d", "max"),
        max_cnt_7d=("cnt_7d", "max"),
        max_scenario_hits=("scenario_hits", "max"),
    )
)

alerts.head()


,sender_account,alert_date,alert_txn_count,alert_amt_sum,alert_unique_receivers,alert_xborder_cnt,alert_cash_cnt,max_cnt_1d,max_cnt_7d,max_scenario_hits
0,61549,2022-12-23,8,39940.85,1,8.0,0,12.0,12.0,1
1,92172,2023-02-08,1,3579.98,1,0.0,0,3.0,7.0,1
2,251764,2023-03-29,8,83397.97,1,8.0,0,12.0,12.0,1
3,344654,2022-11-02,1,9591.64,1,0.0,0,10.0,10.0,1
4,404147,2023-08-07,8,49489.90,1,8.0,0,12.0,12.0,2


### Sanity Check

In [24]:
print(df_feat["any_alert"].mean())
print(alerts.shape)
alerts["max_scenario_hits"].value_counts().head()


0.26834347341757664
(166983, 10)


1    144579
2     12686
3      9681
4        37
Name: max_scenario_hits, dtype: int64

### Stage 2B: Alert enrichment and typology attribution
* Attach dominant laundering typology for analysis
* Not used as a model feature
* Used for QA, tuning, and validation

In [ ]:
pos_typology = (
    df_feat[df_feat["is_laundering"] == 1]
    .groupby(["sender_account", "alert_date"])["laundering_type"]
    .agg(lambda x: x.value_counts().index[0])
    .reset_index()
    .rename(columns={"laundering_type": "dominant_typology"})
)

alerts = alerts.merge(pos_typology, on=["sender_account", "alert_date"], how="left")
alerts["dominant_typology"] = alerts["dominant_typology"].fillna("unknown")
alerts["dominant_typology"].value_counts().head(10)


unknown                 165930
Cash_Withdrawal            445
Smurfing                   317
Behavioural_Change_2       118
Behavioural_Change_1        79
Structuring                 24
Layered_Fan_Out             12
Stacked Bipartite           11
Deposit-Send                10
Fan_Out                      8
Name: dominant_typology, dtype: int64

#### Make Stage 1 more bank realistic by tuning thresholds automatically

Instead of guessing thresholds, we’ll set them using percentiles so only the most extreme behavior triggers.

Run this block. It will rebuild the scenario flags and any_alert.

In [27]:
# choose conservative percentiles
p_cnt_1d = df_feat["cnt_1d"].quantile(0.995)
p_xborder_7d = df_feat["xborder_cnt_7d"].quantile(0.995)
p_cash_7d = df_feat["cash_cnt_7d"].quantile(0.995)

print("cnt_1d 99.5% threshold:", p_cnt_1d)
print("xborder_cnt_7d 99.5% threshold:", p_xborder_7d)
print("cash_cnt_7d 99.5% threshold:", p_cash_7d)

df_feat["scn_high_velocity"] = (df_feat["cnt_1d"] >= p_cnt_1d).astype("int8")
df_feat["scn_xborder_burst"] = (df_feat["xborder_cnt_7d"] >= p_xborder_7d).astype("int8")
df_feat["scn_cash_intensive"] = (df_feat["cash_cnt_7d"] >= p_cash_7d).astype("int8")

# rapid activity can stay fixed
df_feat["scn_rapid_activity"] = (
    df_feat["mins_since_prev"].fillna(9999) <= 2
).astype("int8")

scenario_cols = [
    "scn_high_velocity",
    "scn_xborder_burst",
    "scn_cash_intensive",
    "scn_rapid_activity",
]

df_feat["scenario_hits"] = df_feat[scenario_cols].sum(axis=1).astype("int8")
df_feat["any_alert"] = (df_feat["scenario_hits"] > 0).astype("int8")

print("new any_alert rate:", df_feat["any_alert"].mean())
df_feat[scenario_cols].mean().sort_values(ascending=False)


cnt_1d 99.5% threshold: 232.0
xborder_cnt_7d 99.5% threshold: 107.0
cash_cnt_7d 99.5% threshold: 4.0
new any_alert rate: 0.08793361537875603


scn_rapid_activity    0.072460
scn_cash_intensive    0.009813
scn_high_velocity     0.005138
scn_xborder_burst     0.005015
dtype: float64

#### Rebuild alerts table after tuning

Now rebuild alerts so Stage 2 uses the tuned alert population.

In [29]:
alert_labels = (
    df_feat.groupby(["sender_account", "alert_date"], as_index=False)["is_laundering"]
          .max()
          .rename(columns={"is_laundering": "alert_is_laundering"})
)

alert_labels.head()


,sender_account,alert_date,alert_is_laundering
0,9018,2023-07-22,0
1,28511,2023-01-24,0
2,28511,2023-02-24,0
3,28511,2023-03-24,0
4,28511,2023-04-24,0


In [31]:
df_feat["alert_date"] = df_feat["transaction_datetime"].dt.date

alerts = (
    df_feat[df_feat["any_alert"] == 1]
    .groupby(["sender_account", "alert_date"], as_index=False)
    .agg(
        alert_txn_count=("txn_count", "sum"),
        alert_amt_sum=("amount", "sum"),
        alert_unique_receivers=("receiver_account", "nunique"),
        alert_xborder_cnt=("is_cross_border", "sum"),
        alert_cash_cnt=("is_cash_like", "sum"),
        max_cnt_1d=("cnt_1d", "max"),
        max_cnt_7d=("cnt_7d", "max"),
        max_scenario_hits=("scenario_hits", "max"),
        scn_high_velocity=("scn_high_velocity", "max"),
        scn_xborder_burst=("scn_xborder_burst", "max"),
        scn_cash_intensive=("scn_cash_intensive", "max"),
        scn_rapid_activity=("scn_rapid_activity", "max"),
    )
)

# label again
alerts = alerts.merge(alert_labels, on=["sender_account", "alert_date"], how="left")
alerts["alert_is_laundering"] = alerts["alert_is_laundering"].fillna(0).astype("int8")

print("alerts shape aka alerts sender day level:", alerts.shape)
print("alert positive rate:", alerts["alert_is_laundering"].mean())


alerts shape aka alerts sender day level: (104933, 15)
alert positive rate: 0.010616298018735764


### Stage 2C: Alert Scoring Model
* Train logistic regression / GB model
* Generate risk score
* Rank alerts

1) Time based split


In [32]:
import pandas as pd
import numpy as np

alerts["alert_date"] = pd.to_datetime(alerts["alert_date"])

cutoff = alerts["alert_date"].quantile(0.8)

train = alerts[alerts["alert_date"] <= cutoff].copy()
test = alerts[alerts["alert_date"] > cutoff].copy()

print("cutoff:", cutoff.date())
print("train shape:", train.shape, "test shape:", test.shape)
print("train positive rate:", train["alert_is_laundering"].mean())
print("test positive rate:", test["alert_is_laundering"].mean())


cutoff: 2023-06-18
train shape: (84040, 15) test shape: (20893, 15)
train positive rate: 0.010316515944788196
test positive rate: 0.011822141387067438


2) Build features and baseline logistic regression

In [33]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score

target = "alert_is_laundering"

feature_cols = [
    "alert_txn_count",
    "alert_amt_sum",
    "alert_unique_receivers",
    "alert_xborder_cnt",
    "alert_cash_cnt",
    "max_cnt_1d",
    "max_cnt_7d",
    "max_scenario_hits",
    "scn_high_velocity",
    "scn_xborder_burst",
    "scn_cash_intensive",
    "scn_rapid_activity",
]

X_train = train[feature_cols]
y_train = train[target].astype(int)

X_test = test[feature_cols]
y_test = test[target].astype(int)

pipe_lr = Pipeline(steps=[
    ("scaler", StandardScaler(with_mean=False)),
    ("clf", LogisticRegression(max_iter=400, class_weight="balanced"))
])

pipe_lr.fit(X_train, y_train)
proba_lr = pipe_lr.predict_proba(X_test)[:, 1]

print("LogReg ROC AUC:", roc_auc_score(y_test, proba_lr))
print("LogReg Avg Precision:", average_precision_score(y_test, proba_lr))


LogReg ROC AUC: 0.9056771934530846
LogReg Avg Precision: 0.10656857349119224


3) AML Style evaluation: precision at top KeyboardInterrupt

This is the core operational metric

In [35]:
def precision_at_k(y_true, y_score, k_frac):
    y_true = pd.Series(y_true).reset_index(drop=True)
    y_score = pd.Series(y_score).reset_index(drop=True)
    k = max(1, int(len(y_true) * k_frac))
    top_idx = np.argsort(-y_score.values)[:k]
    return y_true.iloc[top_idx].mean(), k

for k in [0.01, 0.05, 0.10]:
    p, n = precision_at_k(y_test, proba_lr, k)
    print(f"LogReg Precision@Top {int(k*100)}% (n={n}): {p:.4f}")


LogReg Precision@Top 1% (n=208): 0.1538
LogReg Precision@Top 5% (n=1044): 0.1216
LogReg Precision@Top 10% (n=2089): 0.0862


4) Upgrad model: Gradient Boosting (Stronger)

Strong step to take for better ranking performance

In [36]:
from sklearn.ensemble import HistGradientBoostingClassifier

gb = HistGradientBoostingClassifier(
    max_depth=6,
    learning_rate=0.05,
    max_iter=300
)

gb.fit(X_train, y_train)
proba_gb = gb.predict_proba(X_test)[:, 1]

print("GB ROC AUC:", roc_auc_score(y_test, proba_gb))
print("GB Avg Precision:", average_precision_score(y_test, proba_gb))

for k in [0.01, 0.05, 0.10]:
    p, n = precision_at_k(y_test, proba_gb, k)
    print(f"GB Precision@Top {int(k*100)}% (n={n}): {p:.4f}")


GB ROC AUC: 0.9567626592244589
GB Avg Precision: 0.6479214039628729
GB Precision@Top 1% (n=208): 0.6779
GB Precision@Top 5% (n=1044): 0.1753
GB Precision@Top 10% (n=2089): 0.1010


#### Investigator facing output: risk score + reason codes

5) Score alerts and add reason codes

This gives each alert a score and top drivers

In [37]:
test_scored = test.copy()
test_scored["risk_score"] = proba_gb

reason_features = [
    "alert_txn_count",
    "alert_amt_sum",
    "alert_unique_receivers",
    "alert_xborder_cnt",
    "alert_cash_cnt",
    "max_cnt_1d",
    "max_cnt_7d",
    "max_scenario_hits",
]

for col in reason_features:
    test_scored[col + "_pct"] = test_scored[col].rank(pct=True)

def top_reasons(row, n=3):
    pairs = [(c, row[c + "_pct"]) for c in reason_features]
    pairs = sorted(pairs, key=lambda x: x[1], reverse=True)[:n]
    return ", ".join([p[0] for p in pairs])

test_scored["reason_codes"] = test_scored.apply(top_reasons, axis=1)

test_scored[["sender_account", "alert_date", "risk_score", "reason_codes", "alert_is_laundering"]].head(15)


,sender_account,alert_date,risk_score,reason_codes,alert_is_laundering
2,404147,2023-08-07,0.000080,"alert_xborder_cnt, max_cnt_1d, max_scenario_hits",0
7,716444,2023-07-04,0.000112,"alert_unique_receivers, max_cnt_1d, max_scenar...",0
19,2491634,2023-08-23,0.000228,"alert_xborder_cnt, alert_unique_receivers, ale...",0
25,3362699,2023-07-12,0.000103,"alert_amt_sum, max_scenario_hits, alert_xborde...",0
30,4670237,2023-07-20,0.000077,"max_cnt_1d, max_scenario_hits, alert_xborder_cnt",0
31,4693797,2023-08-17,0.000092,"max_cnt_1d, alert_amt_sum, max_cnt_7d",0
43,5038103,2023-07-10,0.011926,"alert_cash_cnt, max_scenario_hits, alert_xbord...",0
44,5038103,2023-07-29,0.011419,"alert_cash_cnt, max_scenario_hits, alert_xbord...",0
45,5038103,2023-07-31,0.016642,"alert_cash_cnt, alert_unique_receivers, alert_...",0
46,5038103,2023-08-03,0.000103,"alert_unique_receivers, alert_txn_count, alert...",0


#### Capacity Simulation but optional

In [38]:
daily_capacity = 300

daily = (
    test_scored.sort_values(["alert_date", "risk_score"], ascending=[True, False])
    .groupby("alert_date")
    .head(daily_capacity)
)

print("Avg positives found per day within capacity:", daily["alert_is_laundering"].mean())
print("Days evaluated:", daily["alert_date"].nunique())


Avg positives found per day within capacity: 0.012787988609888688
Days evaluated: 66


Model Performance Summary

Model performance is evaluated using AML relevant metrics rather than accuracy alone.

Key evaluation criteria include:

Precision at the top 1%, 5%, and 10% of alerts

Ability to concentrate true positives within investigator capacity

Stability of performance across time based splits

The gradient boosted model demonstrates improved alert prioritization while maintaining reasonable interpretability, making it suitable for alert triage use cases.

Baseline model results (logistic regression)

Paste the printed output for:

ROC AUC

Average Precision

Precision@Top 1%, 5%, 10%

In [39]:
print("LogReg ROC AUC:", roc_auc_score(y_test, proba_lr))
print("LogReg Avg Precision:", average_precision_score(y_test, proba_lr))

for k in [0.01, 0.05, 0.10]:
    p, n = precision_at_k(y_test, proba_lr, k)
    print(f"LogReg Precision@Top {int(k*100)}% (n={n}): {p:.4f}")


LogReg ROC AUC: 0.9056771934530846
LogReg Avg Precision: 0.10656857349119224
LogReg Precision@Top 1% (n=208): 0.1538
LogReg Precision@Top 5% (n=1044): 0.1216
LogReg Precision@Top 10% (n=2089): 0.0862


Gradient boosting results

Paste the printed output for:

ROC AUC

Average Precision

Precision@Top 1%, 5%, 10%

In [40]:
print("GB ROC AUC:", roc_auc_score(y_test, proba_gb))
print("GB Avg Precision:", average_precision_score(y_test, proba_gb))

for k in [0.01, 0.05, 0.10]:
    p, n = precision_at_k(y_test, proba_gb, k)
    print(f"GB Precision@Top {int(k*100)}% (n={n}): {p:.4f}")


GB ROC AUC: 0.9567626592244589
GB Avg Precision: 0.6479214039628729
GB Precision@Top 1% (n=208): 0.6779
GB Precision@Top 5% (n=1044): 0.1753
GB Precision@Top 10% (n=2089): 0.1010


Alert Explainability and Reason Codes

To support investigator decision making, each alert is accompanied by reason codes derived from the most extreme contributing risk factors. These reason codes highlight behaviors such as elevated transaction volume, cross border activity, cash intensity, or rapid transaction frequency.

This approach supports transparency and aligns with expectations for explainable AML systems.

Sample scored alerts with reason codes

In [41]:
test_scored[
    ["sender_account", "alert_date", "risk_score", "reason_codes", "alert_is_laundering"]
].head(10)


,sender_account,alert_date,risk_score,reason_codes,alert_is_laundering
2,404147,2023-08-07,0.000080,"alert_xborder_cnt, max_cnt_1d, max_scenario_hits",0
7,716444,2023-07-04,0.000112,"alert_unique_receivers, max_cnt_1d, max_scenar...",0
19,2491634,2023-08-23,0.000228,"alert_xborder_cnt, alert_unique_receivers, ale...",0
25,3362699,2023-07-12,0.000103,"alert_amt_sum, max_scenario_hits, alert_xborde...",0
30,4670237,2023-07-20,0.000077,"max_cnt_1d, max_scenario_hits, alert_xborder_cnt",0
31,4693797,2023-08-17,0.000092,"max_cnt_1d, alert_amt_sum, max_cnt_7d",0
43,5038103,2023-07-10,0.011926,"alert_cash_cnt, max_scenario_hits, alert_xbord...",0
44,5038103,2023-07-29,0.011419,"alert_cash_cnt, max_scenario_hits, alert_xbord...",0
45,5038103,2023-07-31,0.016642,"alert_cash_cnt, alert_unique_receivers, alert_...",0
46,5038103,2023-08-03,0.000103,"alert_unique_receivers, alert_txn_count, alert...",0


Governance and limitations (very important)
Governance Considerations and Limitations

The model is intended for prioritization, not automated decisioning

Thresholds and scenarios require periodic tuning as behavior evolves

Performance should be monitored for data drift and population changes

Model outputs should be reviewed in conjunction with investigator judgment

Ongoing monitoring would include alert volume tracking, precision stability, and periodic challenger model evaluation.